# Scimap neighborhood analysis

## Prerequisites
scimap environment (see https://scimap.xyz/Getting%20Started/)

### Import packages

In [ ]:
import os
import scimap as sm
import anndata as ad
import tempfile
import pandas as pd

### Read Anndata

In [ ]:
OUTPUT_DIR = tempfile.gettempdir()
adata_path = os.path.join(OUTPUT_DIR, 'adata.h5ad')

adata = ad.read_h5ad(adata_path)
adata

### Scimap neighborhood analysis

In [ ]:
cell_type_column = 'annotation'
radius = 150 # radius in pixels

# Get spatial coordinates and add to adata.obs
spatial_coords = adata.obsm["spatial"]
spatial_coords_df = pd.DataFrame(spatial_coords, columns=["x", "y"], index=adata.obs.index)
adata.obs[["x", "y"]] = spatial_coords_df

# Identify neighbors
adata = sm.tl.spatial_count(
    adata, 
    x_coordinate='x',
    y_coordinate='y',
    z_coordinate=None,
    phenotype=cell_type_column, 
    method='radius', 
    radius=radius, 
    imageid='fov_labels',
    label='spatial_count'
)

# Cluster results into neighborhoods
k = 6 # Number of clusters to form
adata = sm.tl.spatial_cluster(
    adata, 
    df_name='spatial_count', 
    method='kmeans', 
    k=k, 
    label=f'neigh_kmeans_{k}'
)

# Plot neighborhoods spatially
sm.pl.spatial_scatterPlot(
    adata, 
    colorBy=f'neigh_kmeans_{k}', 
    topLayer=None, 
    x_coordinate='x',
    y_coordinate='y',
    imageid='region', 
    s=5,
    alpha=1, 
    dpi=300, 
    figsize=(5.8, 5), 
)

In [ ]:
from matplotlib.colors import ListedColormap

colors_33 = [
    "#d95f02", "#7570b3", "#e7298a", "#66a61e", "#e6ab02", "#a6761d",
    "#666666", "#8c564b", "#c49c94", "#e377c2", "#7f7f7f", "#bcbd22",
    "#e41a1c", "#377eb8", "#4daf4a", "#984ea3", "#ff7f00", "#ffff33",
    "#a65628", "#f781bf", "#999999", "#66c2a5", "#fc8d62", "#8da0cb",
    "#e78ac3", "#a6d854", "#ffd92f", "#e5c494", "#b3b3b3", "#1b9e77",
    "#17becf", "#aec7e8", "#ffbb78"
]
cmap_33 = ListedColormap(colors_33)

sm.pl.stacked_barplot(
    adata, 
    x_axis=f'neigh_kmeans_{k}', 
    y_axis='annotation', 
    matplotlib_cmap=cmap_33)